<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/basics/data_rows.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/basics/data_rows.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Data rows

* Data rows are the assets that are being labeled. We currently support the following asset types:
    * Image
    * Text
    * Video
    * Geospatial / Tiled Imagery
    * Audio
    * Documents 
    * HTML 
    * DICOM 
    * Conversational
* A data row cannot exist without belonging to a dataset.
* Data rows are added to labeling tasks by first attaching them to datasets and then creating batches in projects

In [ ]:
!pip install labelbox -q

In [3]:
import labelbox as lb
import uuid
import json

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [4]:
# Add your api key
API_KEY = ""
client = lb.Client(api_key=API_KEY)

### Get data rows from projects

In [5]:
# Pick a project with batches that have data rows with global keys
PROJECT_ID = "<PROJECT-ID>"
project = client.get_project(PROJECT_ID)
batches = list(project.batches())
print(batches)
# This is the same as
# -> dataset = client.get_dataset(dataset_id)

AuthenticationError: Invalid API key('Invalid API key', None)

### Fetch data rows from project's batches

Batches will need to be exported from your project as a export parameter. Before you can export from a project you will need an ontology attached.

In [ ]:
client.enable_experimental = True

batch_ids = [batch.uid for batch in batches]

export_params = {
 "attachments": True,
  "metadata_fields": True,
  "data_row_details": True,
  "project_details": True,
  "performance_details": True,
  "batch_ids" : batch_ids # Include batch ids if you only want to export specific batches, otherwise,
  #you can export all the data without using this parameter
}
filters = {}

# A task is returned, this provides additional information about the status of your task, such as
# any errors encountered
export_task = project.export(params=export_params, filters=filters)
export_task.wait_till_done()

In [ ]:
data_rows = []

def json_stream_handler(output: lb.JsonConverterOutput):
  data_row = json.loads(output.json_str)
  data_rows.append(data_row)


if export_task.has_errors():
  export_task.get_stream(
  converter=lb.JsonConverter(),
  stream_type=lb.StreamType.ERRORS
  ).start(stream_handler=lambda error: print(error))

if export_task.has_result():
  export_json = export_task.get_stream(
    converter=lb.JsonConverter(),
    stream_type=lb.StreamType.RESULT
  ).start(stream_handler=json_stream_handler)

In [ ]:
# Get single data row
data_row = data_rows[0]
print(data_row)

### Get labels from the data row

In [ ]:
print("Associated label(s)", data_row["projects"][project.uid]["labels"])
print("Global key", data_row["data_row"]["global_key"])

### Get data row ids by using global keys

In [ ]:
global_key = "<ENTER GLOBAL KEY>"
task = client.get_data_row_ids_for_global_keys([global_key])
print(f"Data row id: {task['results']}")

## Create
We recommend the following methods to create data rows : `dataset.upsert_data_rows()`, and `dataset.create_data_rows()`, 

### Create data rows via `dataset.upsert_data_rows()`

In [ ]:
# Create a dataset
dataset = client.create_dataset(name="data_rows_demo_dataset_6")
# You can also upload metadata along with your data row
mdo = client.get_data_row_metadata_ontology()

In [ ]:
uploads = []
# Generate data rows
for i in range(1,8):
    uploads.append({
        "row_data":  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key": "TEST-ID-%id" % uuid.uuid1(),
        ## add metadata (optional)
        "metadata_fields": [
            lb.DataRowMetadataField(
                schema_id=mdo.reserved_by_name["tag"].uid,  # specify the schema id
                value="tag_string", # typed inputs
            ),
        ],
        "attachments": [
            {
                "type": "IMAGE_OVERLAY",
                "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/disease_attachment.jpeg"
            },
            {
                "type": "RAW_TEXT",
                "value": "IOWA, Zone 2232, June 2022 [Text string]"
            },
            {
                "type": "TEXT_URL",
                "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/text_attachment.txt"
            },
            {
                "type": "IMAGE",
                "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/disease_attachment.jpeg"
            },
            {
                "type": "VIDEO",
                "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/drone_video.mp4"
            },
            {
                "type": "HTML",
                "value": "https://storage.googleapis.com/labelbox-sample-datasets/Docs/windy.html"
            },
            {
                "type": "PDF_URL",
                "value": "https://storage.googleapis.com/labelbox-datasets/arxiv-pdf/data/99-word-token-pdfs/0801.3483.pdf"
            }
        ]
    })

task1 = dataset.upsert_data_rows(uploads)
task1.wait_till_done()
print("ERRORS: " , task1.errors)
print("RESULTS:" , task1.result)

Create data rows from data in your local path 

In [ ]:
from PIL import Image

# Create dummy empty jpeg file
width = 400
height = 300
color = (255, 255, 255)  # White color
image = Image.new("RGB", (width, height), color)

# Save the image as a JPEG file
image.save("dummy.jpg")

local_data_path = "dummy.jpg"

data = {
  "row_data" : local_data_path,
  "global_key": str(uuid.uuid4())
}

task3 = dataset.upsert_data_rows([data])
task3.wait_till_done()
print("ERRORS: " , task3.errors)
print("RESULTS:" , task3.result)

In [ ]:
# You can mix local files with urls when creating data rows
task4 = dataset.upsert_data_rows([{
    "row_data": "https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_0009.jpeg",
    "global_key": str(uuid.uuid4())
    }, {
    "row_data": local_data_path,
    "global_key": str(uuid.uuid4())
    }])
task4.wait_till_done()
print("ERRORS: " , task4.errors)
print("RESULTS:" , task4.result)

### Create data rows via `dataset.create_data_rows()`


In [ ]:
dataset_2 = client.create_dataset(name="data_rows_demo_dataset_3")

In [ ]:
uploads = []
# Generate data rows
for i in range(1,9):
    uploads.append({
        "row_data":  f"https://storage.googleapis.com/labelbox-datasets/People_Clothing_Segmentation/jpeg_images/IMAGES/img_000{i}.jpeg",
        "global_key": "TEST-ID-%id" % uuid.uuid1(),
        ## add metadata (optional)
        "metadata_fields": [
            lb.DataRowMetadataField(
                schema_id=mdo.reserved_by_name["tag"].uid,  # specify the schema id
                value="tag_string", # typed inputs
            ),
        ]
    })

task1_2 = dataset_2.create_data_rows(uploads)
task1_2.wait_till_done()
print("ERRORS: " , task1_2.errors)
print("RESULTS:" , task1_2.result)

### Update
`dataset.upsert_data_rows()` can also be use to update data rows

To update data rows using this method, you need to pass a `key`, which can reference either a global key or a data row ID. Additionally, include any fields that you wish to update along with their new values.


In [ ]:
# Fetch a data row from the first dataset example
ts = dataset.export()
ts.wait_till_done()
DATA_ROW_ID = [json.loads(output.json_str) for output in ts.get_stream()][0]['data_row']['id']
GLOBAL_KEY = [json.loads(output.json_str) for output in ts.get_stream()][0]['data_row']['global_key']

print(f"Pick either a data row id : {DATA_ROW_ID} or global key: {GLOBAL_KEY}")


In [ ]:
# Update the global key assodicated with the DATAROW_ID or GLOBAL_KEY, and include a additional metadata
data = {
    "key": lb.UniqueId(DATA_ROW_ID),
    "global_key": "NEW-ID-%id" % uuid.uuid1(),
    "metadata_fields": [
        # New metadata
        lb.DataRowMetadataField(
            schema_id=mdo.reserved_by_name['captureDateTime'].uid,
            value="2000-01-01 00:00:00"
        ),
        # Include original metadata otherwise it will be removed
        lb.DataRowMetadataField(
            schema_id=mdo.reserved_by_name["tag"].uid,
            value="tag_string",
        ),
    ]
}

task5 = dataset_2.upsert_data_rows([data])
task5.wait_till_done()
print("ERRORS: " , task5.errors)
print("RESULTS:" , task5.result)

### Create a single attachment on an existing data row

In [ ]:
# You can only create one attachment at the time.
DATA_ROW_ID = "<DATA-ROW-ID>"
data_row = client.get_data_row(DATA_ROW_ID)
attachment = data_row.create_attachment(attachment_type="RAW_TEXT",
                           attachment_value="LABELERS WILL SEE THIS")

Update a recently created attachment 

In [ ]:
attachment.update(type= "RAW_TEXT", value="NEW RAW TEXT")

### Delete

* Delete a single data row

In [ ]:
DATAROW_ID_TO_DELETE = "<DATA-ROW-ID>"
data_row = client.get_data_row(DATAROW_ID_TO_DELETE)
data_row.delete()

* Bulk delete data row objects

In [ ]:
# Bulk delete a list of data_rows ( limit: 4K data rows per call)
lb.DataRow.bulk_delete(list(dataset.data_rows()))